In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd

# Define the paths to the two slice folders
data_dir1 = '/kaggle/input/brats-2020-slices/slices-20230925T215308Z-001/slices'
data_dir2 = '/kaggle/input/brats-2020-slices/slices-20230925T215308Z-002/slices'

# Get a list of image file names from both folders
image_files1 = os.listdir(os.path.join(data_dir1, 'images'))
image_files2 = os.listdir(os.path.join(data_dir2, 'images'))

# Initialize empty lists to store image and mask paths
image_paths = []
mask_paths = []

# Loop through image files from slices1 and check for corresponding masks
for image_file in image_files1:
    image_path = os.path.join(data_dir1, 'images', image_file)
    mask_file = image_file.replace("_image.npy", "_mask.npy")
    mask_path = os.path.join(data_dir1, 'masks', mask_file)
    
    image_paths.append(image_path)
    # Check if the corresponding mask file exists in slices1->masks
    if os.path.exists(mask_path):
        #image_paths.append(image_path)
        mask_paths.append(mask_path)
    else:
        #image_paths.append(image_path)
        mask_path = os.path.join(data_dir2, 'masks', mask_file)
        mask_paths.append(mask_path)
        

# Loop through image files from slices2 and check for corresponding masks
for image_file in image_files2:
    image_path = os.path.join(data_dir2, 'images', image_file)
    mask_file = image_file.replace("_image.npy", "_mask.npy")
    mask_path = os.path.join(data_dir2, 'masks', mask_file)
    
    image_paths.append(image_path)
    # Check if the corresponding mask file exists in slices2->masks
    if os.path.exists(mask_path):
        #image_paths.append(image_path)
        mask_paths.append(mask_path)
    else:
       # image_paths.append(image_path)
        mask_path = os.path.join(data_dir1, 'masks', mask_file)
        mask_paths.append(mask_path)
        
        

# Create a DataFrame to store the paths
data = pd.DataFrame({'image_path': image_paths, 'mask_path': mask_paths})


In [ ]:
# Function to check if a mask has less than 1% positive pixels
import numpy as np
def has_few_positive_pixels(mask_path, threshold=0.05):
    mask = np.load(mask_path)
    positive_pixels = np.sum(mask[:,:,1] > 0)
    total_pixels = mask.size
    return (positive_pixels / total_pixels) < threshold

def has_positive_pixels(mask_path, threshold=0.05):
    mask = np.load(mask_path)
    positive_pixels = np.sum(mask[:,:,0] > 0)
    if positive_pixels>0 :
        return True
    else :
        return False
# Apply the function to the mask_path column
data['has_few_positive_pixels'] = data['mask_path'].apply(has_positive_pixels)

# Filter rows based on the condition
data = data[~data['has_few_positive_pixels']]

# Reset the index if needed
data.reset_index(drop=True, inplace=True)

# Remove the temporary column
data.drop('has_few_positive_pixels', axis=1, inplace=True)


In [ ]:
len(data['image_path'])

In [ ]:
print(data['image_path'][1])
print(data['mask_path'][1])

In [ ]:
# Function to plot images and masks
import matplotlib.pyplot as plt
def plot_images_masks(images, masks):
    num_samples = images.shape[0]

    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 10))
    for i in range(num_samples):
        axes[i, 0].imshow(images[i],cmap='bone',interpolation='bicubic')
        axes[i, 0].set_title('Image')
        axes[i, 0].axis('on')

        axes[i, 1].imshow(masks[i],cmap='bone',interpolation='bicubic')
        axes[i, 1].set_title('Mask')
        axes[i, 1].axis('on')

        print(np.max(images[i]),np.max(masks[i]),masks[i].shape)

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generator parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant'
)

# Function to load and preprocess images and masks
def load_and_preprocess(paths):
    images = []
    masks = []
    for img_path, mask_path in zip(paths['image_path'], paths['mask_path']):
        img = np.load(img_path).astype(np.float32)
        mask = np.load(mask_path).astype(np.float32)
        """mask[:,:,0]=(mask[:,:,0]>0).astype(int)
        mask[:,:,1]=(mask[:,:,1]>0).astype(int)
        mask[:,:,2]=(mask[:,:,2]>0).astype(int)
        new_channel = np.zeros((128, 128, 1), dtype=np.float32)
        mask_4_channel = np.concatenate((new_channel,mask), axis=-1)"""
        mask_=(mask[:,:,1]>0).astype(int)
        mask_=np.expand_dims(mask_,axis=-1)
        
        images.append(img)
        masks.append(mask_)
    return np.array(images), np.array(masks)

# Function to apply augmentations to both images and masks
def apply_augmentation(images, masks):
    seed = np.random.randint(1, 1000)  # Generate a random seed for augmentations
    image_gen = datagen.flow(images, batch_size=len(images), seed=seed)
    mask_gen = datagen.flow(masks, batch_size=len(masks), seed=seed)
    return next(image_gen), next(mask_gen)

# Generate augmented batches of data with synchronized augmentations
def generate_data_generator(data, batch_size):
    while True:
        batch_indices = np.random.choice(len(data), batch_size)
        batch_data = data.iloc[batch_indices]
        x, y = load_and_preprocess(batch_data)
        x_augmented, y_augmented = apply_augmentation(x, y)
        yield x_augmented, y_augmented

# Example usage
batch_size = 4
train_data = generate_data_generator(data, batch_size)

# Plot the batch
batch_images, batch_masks = next(train_data)
plot_images_masks(batch_images, batch_masks)


In [ ]:
epsilon = 1e-5
smooth = 1

def iou(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    class_num = 2
    for i in range(class_num):
        targets_f= K.flatten(targets[:,:,:,i])
        inputs_f = K.flatten(inputs[:,:,:,i])
       

        intersection = K.sum((targets_f*inputs_f))
        total = K.sum(targets_f) + K.sum(inputs_f)
        union = total - intersection
        loss = (intersection + smooth) / (union + smooth)
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
    return total_loss

def iou_tc(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    targets_f = K.flatten(targets[:,:,:,1])
    inputs_f = K.flatten(inputs[:,:,:,1])
    
    intersection = K.sum((targets_f * inputs_f))
    total = K.sum(targets_f) + K.sum(inputs_f)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return IoU

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def focal_tversky(y_true,y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

from keras import backend as K

def dice_coeff(y_true, y_pred, smooth=1):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    intersection = K.sum(y_true_flatten * y_pred_flatten)
    dice = (2. * intersection + smooth) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) + smooth)
    return dice
def dice_loss(y_true, y_pred, smooth=1):
    return  1- dice_coeff(y_true, y_pred, smooth=1)

def FocalLoss(targets, inputs, alpha=0.8, gamma=2):    
    
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss

def total_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return dice_loss(y_true, y_pred, smooth=1) + FocalLoss(y_true, y_pred, 0.25,2)

def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
from tensorflow.keras.layers import Conv2D,Conv2DTranspose, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Input, ZeroPadding2D,Dropout,Dense,MaxPooling2D,Reshape,Multiply,GlobalAveragePooling2D,AveragePooling2D,Lambda
from tensorflow.keras.models import Model
from sklearn.decomposition import NMF

def batchnorm_relu(inputs):
    x = BatchNormalization(axis=-1)(inputs)
    x = Activation("relu")(x)
    return x
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=3, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=5, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=7, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y


def GCSE(input_tensor, ratio=8, i=0):
    
  
    # Compute the global statistics (mean and std deviation) along the channel axis.
    mean = tf.reduce_mean(input_tensor, axis=[1, 2], keepdims=True)
    std = tf.math.reduce_std(input_tensor, axis=[1, 2], keepdims=True)
    
    # Compute channel-wise attention using a convolutional neural network.
    attn_channel = tf.concat([mean, std], axis=-1)
    attn_channel = tf.keras.layers.Conv2D(filters=input_tensor.shape[-1] // ratio, kernel_size=(1, 1), activation='relu',name="first_channel_attention_"+str(i))(attn_channel)
    attn_channel = tf.keras.layers.Conv2D(filters=input_tensor.shape[-1], kernel_size=(1, 1), activation='sigmoid',name="second_channel_attention_"+str(i))(attn_channel)
    
    # Compute spatial attention using global information from the input tensor.
    global_info = tf.reduce_mean(input_tensor, axis=-1, keepdims=True)
    attn_spatial = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid',name="spatial_attention_"+str(i))(global_info)
    
    # Combine channel-wise and spatial attention.
    attn = tf.keras.layers.Multiply(name='combined_attention_'+str(i))([attn_channel,attn_spatial])
    
    # Multiply the input tensor by the learned attention weights.
    output_tensor = tf.keras.layers.Multiply(name='SE_out_'+str(i))([input_tensor,attn])
    
    return output_tensor

def residual_block(inputs, num_filters):
    """ Convolutional Layer """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=1,kernel_initializer = "he_normal")(x)
    x = Dropout(0.1)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1,kernel_initializer = "he_normal")(x)
    #x = squeeze_excite_block(x,8)

    """ Shortcut Connection """
    s = Conv2D(num_filters, 1, padding="same", strides=1,kernel_initializer="he_normal")(inputs)
    x = x + s
    return x

def decoder_block(inputs, skip_features, num_filters,i):
    x = Conv2DTranspose(num_filters,2, strides=(2,2), kernel_initializer="he_normal",padding = "same")(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block(x, num_filters)
    x = GCSE(x,i=i)
    return x

def build_resunet(input_shape):
    inputs = Input(input_shape)
    
    #integrating augmentation directly into the model
    #x = tf.keras.layers.RandomContrast(0.15)(inputs)
    #x = tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2)(x)
    #x = tf.keras.layers.RandomFlip("horizontal_and_vertical")(x)
    #x_ = tf.keras.layers.RandomZoom(0.3)(x)
    
   

    

    """ Encoder 1 """
    x = Conv2D(32, 3, padding="same", strides=1,kernel_initializer="he_normal")(inputs)
    x = batchnorm_relu(x)
    x = Conv2D(32, 3, padding="same", strides=1,kernel_initializer="he_normal")(x)
    s = Conv2D(32, 1, padding="same", strides=1,kernel_initializer="he_normal")(inputs)
    
    c1 = x + s
    c1 =GCSE(c1,i=1)
    p1 = MaxPooling2D((2,2))(c1)
    #s1 = residual_block(c1,32)
    s1 = c1

    """ Encoder 2 and 3 """
    c2 = residual_block(p1, 64)
    c2 = GCSE(c2,i=2)
    p2 = MaxPooling2D((2,2))(c2)
    #s2 = residual_block(c2,64)
    s2 = c2
    
    c3 = residual_block(p2, 128)
    c3 =GCSE(c3,i=4)
    p3 = MaxPooling2D((2,2))(c3)
    #s3 = residual_block(c3,128)
    s3 = c3

    c4 = residual_block(p3, 256)
    c4 = GCSE(c4,i=5)
    p4 = MaxPooling2D((2,2))(c4)
    #s4 = residual_block(c4,256)
    s4 = c4

    """ Bridge """
    b = ASPP(p4,256)
    #b = p4

    """ Decoder 1, 2, 3 """
    d1 = decoder_block(b, s4, 256,i=6)
    d2 = decoder_block(d1, s3, 128,i=7)
    d3 = decoder_block(d2, s2, 64,i=8)
    d4 = decoder_block(d3, s1, 32,i=9)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs)
    return model


In [ ]:
!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

In [ ]:
# compling model and callbacks functions
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.callbacks import CSVLogger
model = build_resunet((128,128,3))
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = adam, 
                  loss = total_loss, 
                  metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5),dice_coeff,precision,sensitivity,specificity,tf.keras.metrics.AUC()]
                 )
#callbacks
csv_logger = CSVLogger('/kaggle/working/training_log_tc', separator=',', append=False)
"""earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=8
                             )"""
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="/kaggle/working/seg_model_tc.h5", 
                               verbose=1, 
                               save_best_only=True,
                               save_weights_only=True
                              )
                              
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=3,
                              min_delta=0.000001,
                              factor=0.2
                             )
callbacks = [checkpointer, reduce_lr, csv_logger]

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.20, random_state=42)

# Define batch size
batch_size = 8

# Create data generators for training and validation sets
train_generator = generate_data_generator(train_data, batch_size)
val_generator = generate_data_generator(val_data, batch_size)

train_steps_per_epoch = len(train_data) // batch_size
val_steps_per_epoch = len(val_data) // batch_size


In [ ]:
history = model.fit(train_generator, 
                  steps_per_epoch=train_steps_per_epoch,
                  epochs = 50, 
                  validation_data = val_generator,
                  validation_steps=val_steps_per_epoch,
                  callbacks = callbacks
                 )

In [ ]:
history = pd.read_csv('/kaggle/input/brats-slices-0-05-model-and-log-tc/training_log_tc', sep=',', engine='python')

#hist=history

############### ########## ####### #######

iou_score=history.history['iou_score']
val_iou_score=history.history['val_iou_score']

loss=history.history['loss']
val_loss=history.history['val_loss']

train_dice=history.history['dice_coeff']
val_dice=history.history['val_dice_coeff']

epoch = np.arange(len(loss))

# Create subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.subplots_adjust(wspace=0.3)  # Adjust the space between subplots

# Plot 1: Loss
axes[0].plot(epoch, loss, 'b', label='Training Loss', linewidth=1.5)
axes[0].plot(epoch, val_loss, 'r', label='Validation Loss', linewidth=1.5)
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()

# Plot 2: IOU Scores
axes[1].plot(epoch, iou_score, 'b', label='Training IOU', linewidth=1.5)
axes[1].plot(epoch, val_iou_score, 'r', label='Validation IOU', linewidth=1.5)
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('IOU Score')
axes[1].set_title('Training and Validation IOU Score')
axes[1].legend()

# Plot 3: Dice Coefficients
axes[2].plot(epoch, train_dice, 'b', label='Training Dice Coeff', linewidth=1.5)
axes[2].plot(epoch, val_dice, 'r', label='Validation Dice Coeff', linewidth=1.5)
axes[2].set_xlabel('Epochs')f
axes[2].set_ylabel('Dice Coefficient')
axes[2].set_title('Training and Validation Dice Coefficient')
axes[2].legend()

# Customize tick marks and labels
for ax in axes:
    ax.grid(True)
    ax.set_xticks(np.arange(0, len(epoch), 5))  # Adjust the x-axis ticks
    ax.set_yticks(np.arange(0, 1.1, 0.1))  # Adjust the y-axis ticks
    ax.grid(which='both', linestyle='--', linewidth=0.5)

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
for layer in model.layers:
    print(layer.name)

In [ ]:
def generate_data_generator(data, batch_size):
    while True:
        batch_indices = np.random.choice(len(data), batch_size)
        batch_data = data.iloc[batch_indices]
        x, y = load_and_preprocess(batch_data)
        #x_augmented, y_augmented = apply_augmentation(x, y)
        #yield x_augmented, y_augmented
        yield x,y

In [ ]:
val_generator = generate_data_generator(val_data, batch_size)

In [ ]:
sample = next(val_generator)

In [ ]:
sample[0][0:1].shape

In [ ]:
# Extract the input and the corresponding mask from the sample
input_sample = sample[0][0:1]  # Assuming the input is the first element in the sample
mask_sample = sample[1][0:1]  # Assuming the mask is the second element in the sample

# Predict the output using the trained model
output = model.predict(input_sample)>0.7

# Plot the input, mask, and output
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title('Input')
plt.imshow(np.squeeze(input_sample), cmap='gray')  # Assuming the input is grayscale
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Mask')
plt.imshow(np.squeeze(mask_sample), cmap='gray')  # Assuming the mask is grayscale
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Output')
plt.imshow(np.squeeze(output), cmap='gray')  # Assuming the output is grayscale
plt.axis('off')

plt.show()

In [ ]:
channel_wise_attention_model = Model(inputs=model.input, outputs=model.get_layer('second_channel_attention_2').output)
spatial_attention_model = Model(inputs=model.input, outputs=model.get_layer('spatial_attention_2').output)
combined_attention_model = Model(inputs=model.input, outputs=model.get_layer('combined_attention_2').output)
SE_out_model = Model(inputs=model.input, outputs=model.get_layer('SE_out_2').output)

In [ ]:
SE_out = SE_out_model.predict(input_sample)

In [ ]:
plt.imshow(SE_out[0,:,:,5])

In [ ]:
SE_out.shape

In [ ]:
feature_map = SE_out

In [ ]:

if len(feature_map.shape) == 4:
    k = feature_map.shape[-1]
    size=feature_map.shape[1]
    image_belt = np.zeros((size,size*k))
    for i in range(k//2):
        feature_image = feature_map[0, :, :, i]
        feature_image-= feature_image.mean()
        feature_image/= feature_image.std ()
        feature_image*=  64
        feature_image+= 128
        feature_image= np.clip(feature_image, 0, 255).astype('uint8')
        image_belt[:,i * size : (i + 1) * size] = feature_image    


scale = 100. / k
plt.figure( figsize=(scale * k, scale) )
plt.grid  ( False )
plt.imshow( image_belt, aspect='auto')

In [ ]:
feature_map = spatial_attention_model.predict(input_sample)

In [ ]:
feature_map.shape

In [ ]:
def process(feature_image):
    feature_image-= feature_image.mean()
    feature_image/= feature_image.std ()
    feature_image*=  64
    feature_image+= 128
    feature_image= np.clip(feature_image, 0, 255).astype('uint8')
    return feature_image[0,:,:,:]
plt.imshow(process(feature_map))

In [ ]:
comb_features  = combined_attention_model.predict(input_sample)

In [ ]:
comb_features.shape

In [ ]:
feature_map = comb_features
if len(feature_map.shape) == 4:
    k = feature_map.shape[-1]
    size=feature_map.shape[1]
    image_belt = np.zeros((size,size*k))
    for i in range(k//2):
        feature_image = feature_map[0, :, :, i]
        feature_image-= feature_image.mean()
        feature_image/= feature_image.std ()
        feature_image*=  64
        feature_image+= 128
        feature_image= np.clip(feature_image, 0, 255).astype('uint8')
        image_belt[:,i * size : (i + 1) * size] = feature_image    

scale = 100. / k
plt.figure( figsize=(scale * k, scale) )
plt.grid  ( False )
plt.imshow( image_belt, aspect='auto')

In [ ]:
feature_map= channel_wise_attention_model.predict(input_sample)

In [ ]:
feature_map.shape

In [ ]:
plt.plot(feature_map[0,0,0,:])

In [ ]:
import tensorflow as tf

def novel_se_block(input_tensor, ratio=4):
    """
    Novel Squeeze-and-Excite Block with Dynamic Channel-wise and Spatial Attention.
    
    Args:
        input_tensor: Input tensor to the SE block.
        ratio: Reduction ratio for the hidden layers.
    
    Returns:
        Output tensor with dynamic feature scaling.
    """
    
    # Compute the global statistics (mean and std deviation) along the channel axis.
    mean = tf.reduce_mean(input_tensor, axis=[1, 2], keepdims=True)
    std = tf.math.reduce_std(input_tensor, axis=[1, 2], keepdims=True)
    
    # Compute channel-wise attention using a convolutional neural network.
    attn_channel = tf.concat([mean, std],axis=2)
    print(attn_channel.shape)
    attn_channel = tf.keras.layers.Conv2D(filters=input_tensor.shape[-1] // ratio, kernel_size=(1, 1), activation='relu')(attn_channel)
    attn_channel = tf.keras.layers.Conv2D(filters=input_tensor.shape[-1], kernel_size=(1, 1), activation='sigmoid')(attn_channel)
    
    # Compute spatial attention using a convolutional neural network.
    attn_spatial = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(input_tensor)
    
    # Combine channel-wise and spatial attention.
    attn = attn_channel * attn_spatial
    
    # Multiply the input tensor by the learned attention weights.
    output_tensor = input_tensor * attn
    
    return output_tensor

In [ ]:
input_tensor = tf.keras.layers.Input(shape=(64, 64, 64))  # Example input shape
se_output = novel_se_block(input_tensor)